- step 1: take the abstract and answer the qa or create the query directly from abstract 
- step 2: get top 30 recommentations 
- step 3: find the cocitations of the rec to the parent 
- step 4: calculate the relevance 
- step 5: calculate the novelty 
- step 6: calculate the important key words or title keywords 
- step 7: create eval metrics 
- step 8: creates graph and thing like that 



Evaluation Matrix Calculation 
---
- 3*cij cocitation of parent to its recommentation
- 5*rij relevancy of important keywords in the doc
- 2*aij relevancy score of the parent and recommentation 

##### Things to do 
    - Create evaluation matrix 
    - Create calculate the evaluation 
    - Create some model and compare 
    - Create a good embedding from good embedding model and only take hep-th as fields
    - Create LSTM Network Improve the quality of recommentation 

To calculate the relevance TfIdf is a best option which would give me more importance to the relevant keywords found in the text
- cij -> 3 rij -> 5 -> nij -> 1

In [2]:
import json
import os 
import requests
import joblib
import numpy as np 
import torch
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone
import pandas as pd
import re
import pickle 
from dotenv import load_dotenv
load_dotenv()

True

###                                 Cocitation Count function


In [3]:
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import time

def get_api_response(url, max_retries=3, timeout=10, backoff_factor=1):
    session = requests.Session()
    
    # Define retry strategy with backoff factor
    retry_strategy = Retry(
        total=max_retries,
        status_forcelist=[429, 500, 502, 503, 504],
        allowed_methods=["HEAD", "GET", "OPTIONS", "POST"],
        backoff_factor=backoff_factor  # Wait time between retries
    )
    
    adapter = HTTPAdapter(max_retries=retry_strategy)
    session.mount("http://", adapter)
    session.mount("https://", adapter)

    try:
        response = session.get(url, timeout=timeout)
        response.raise_for_status()  # Raises HTTPError for bad responses (4xx, 5xx)
        return response
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

def get_citations(paper1, paper2, max_retries=3, timeout=10, backoff_factor=1):
    size = 20
    c_url1 = os.path.join('https://inspirehep.net/api/arxiv/', paper1)
    c_url2 = os.path.join('https://inspirehep.net/api/arxiv/', paper2)
    
    response1 = get_api_response(c_url1, max_retries=max_retries, timeout=timeout, backoff_factor=backoff_factor)
    response2 = get_api_response(c_url2, max_retries=max_retries, timeout=timeout, backoff_factor=backoff_factor)

    if response1 is None or response2 is None:
        time.sleep(15)
        return 'Not_Available'

    response1 = response1.json()
    response2 = response2.json()
    
    c1 = response1['metadata']['control_number']
    c2 = response2['metadata']['control_number']    
    
    url = f'https://inspirehep.net/api/literature?q=(refersto:recid:{c1}) and (refersto:recid:{c2})'
    response = get_api_response(url, max_retries=max_retries, timeout=timeout, backoff_factor=backoff_factor)
    
    if response is None:
        return 'Not_Available'
    
    data = response.json()
    total = data['hits']['total']
    return total


In [4]:
if 'GOOGLE_API_KEY' not in os.environ:
    os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')

api_key = os.getenv('PINECONE_API_KEY')

if 'PINECONE_API_KEY' not in os.environ:
    os.environ['PINECONE_API_KEY'] = api_key
pc = Pinecone(api_key=api_key)
index_name = 'recomentation'
index = pc.Index(index_name)

In [5]:
def preprocess_abstract(text):
    clean_text = ' '.join(re.findall(r'[a-zA-Z.]{2,}', text))
    return clean_text

def create_query(abstract):
    'Returns a Query from the abstract'
    abstract = preprocess_abstract(abstract)
    model = ChatGoogleGenerativeAI(model='gemini-pro', convert_system_message_to_human=True)

    # few shot prompting + chain of thought 
    template = ChatPromptTemplate.from_messages([
        ('system',
            '''You are a detailed query bot used for creating the best queries for recommendations of research papers.
            your Job is to create queries from the abstract of the reaserch paper

            Steps to Follow: 
            * find the problem or the domain of the reaserch paper 
            * find the key words and key idead the reaserch paper is talking about 
            * create a query based on these thing and avoid making it too much specific 
            * create the query in the format
            Query: <query_sentence>
            
            Examples: 
            We describe a new algorithm, the $(k,\ell)$-pebble game with colors, and use
            it obtain a characterization of the family of $(k,\ell)$-sparse graphs and
            algorithmic solutions to a family of problems concerning tree decompositions of
            graphs. Special instances of sparse graphs appear in rigidity theory and have
            received increased attention in recent years. In particular, our colored
            pebbles generalize and strengthen the previous results of Lee and Streinu and
            give a new proof of the Tutte-Nash-Williams characterization of arboricity. We
            also present a new decomposition that certifies sparsity based on the
            $(k,\ell)$-pebble game with colors. Our work also exposes connections between
            pebble game algorithms and previous sparse graph algorithms by Gabow, Gabow and
            Westermann and Hendrickson
            A: efficient algorithms for sparse graph characterization and decomposition techniques

            
            The evolution of Earth-Moon system is described by the dark matter field
            fluid model proposed in the Meeting of Division of Particle and Field 2004,
            American Physical Society. The current behavior of the Earth-Moon system agrees
            with this model very well and the general pattern of the evolution of the
            Moon-Earth system described by this model agrees with geological and fossil
            evidence. The closest distance of the Moon to Earth was about 259000 km at 4.5
            billion years ago, which is far beyond the Roche's limit. The result suggests
            that the tidal friction may not be the primary cause for the evolution of the
            Earth-Moon system. The average dark matter field fluid constant derived from
            Earth-Moon system data is 4.39 x 10^(-22) s^(-1)m^(-1). This model predicts
            that the Mars's rotation is also slowing with the angular acceleration rate
            about -4.38 x 10^(-22) rad s^(-2).
            Query: models for the evolution of the Earth-Moon system and dark matter influences on planetary motion


            In present paper we propose seemingly new method for finding solutions of
            some types of nonlinear PDEs in closed form. The method is based on
            decomposition of nonlinear operators on sequence of operators of lower orders.
            It is shown that decomposition process can be done by iterative procedure(s),
            each step of which is reduced to solution of some auxiliary PDEs system(s) for
            one dependent variable. Moreover, we find on this way the explicit expression
            of the first-order PDE(s) for first integral of decomposable initial PDE.
            Remarkably that this first-order PDE is linear if initial PDE is linear in its
            highest derivatives.
            The developed method is implemented in Maple procedure, which can really
            solve many of different order PDEs with different number of independent
            variables. Examples of PDEs with calculated their general solutions demonstrate
            a potential of the method for automatic solving of nonlinear PDEs
            Query: methods for solving nonlinear partial differential equations using operator decomposition and iterative procedures'''),
       
        MessagesPlaceholder(variable_name="messages"),
    ])

    parser = StrOutputParser()
    chain = template | model | parser

    messages = [HumanMessage(abstract)]
    ai_message = chain.invoke({'messages': messages})
    if ai_message.startswith('Query:'):
        ai_message = ai_message.replace('Query:', '').strip()
    return ai_message

In [6]:

def get_db(index):
    '''Returs the loaded vectorstore'''
    db = PineconeVectorStore(index=index, embedding=GoogleGenerativeAIEmbeddings(model='models/embedding-001'))
    return db


def get_most_cited(n, category='hep-th'):
    '''Retruns List of dic with id and abstracts'''

    url = f'https://inspirehep.net/api/literature?sort=mostcited&size={n}&arxiv_categories={category}'
    most_cited = []
    response = requests.get(url)
    data = response.json()

    for hit in data['hits']['hits']:
        if hit['metadata']['arxiv_eprints'][0]['categories'][0] == category:
            arxiv_id = hit['metadata']['arxiv_eprints'][0]['value']
            title = hit['metadata']['titles'][0]['title']
            most_cited.append({'id': arxiv_id, 'abstract': hit['metadata']['abstracts'][0]['value'], 'title': title})
    return most_cited


class RelevanceScorer:
    def __init__(self):
        with open('data/tfidf_vectorizer.pkl', 'rb') as file:
            self.vectorizer = pickle.load(file)
        
    def calculate_relevance_scores(self, query, result):
        query_vector = self.vectorizer.transform(query)
        result_vector = self.vectorizer.transform(result)
        cosine = cosine_similarity(query_vector, result_vector).reshape(-1)
        return  cosine

In [12]:
def get_citation_scores(parent_paper, recommended_papers):
    # Not_Available
    citation_scores = []
    for paper in recommended_papers:
        paper_citation = get_citations(parent_paper['id'], paper['id'])
        if paper_citation == 'Not_Available':
            paper_citation = 0
        citation_scores.append(paper_citation)
    return np.array(citation_scores).reshape(-1,)

def context_relevancy(query, recommendations):
    embeddings = GoogleGenerativeAIEmbeddings(model='models/embedding-001')
    query_vec = embeddings.embed_query(query)
    documents = embeddings.embed_documents(recommendations)
    similarities = cosine_similarity([query_vec], documents)
    return similarities

In [23]:

def get_scores(file_path, name):
    '''Parameters: file_path: path of the most_cited and its recommentations (torch format)
                   name: name of the recommentation technique'''
    k_rel_scorer = RelevanceScorer()
    
    recommentations = torch.load(file_path)
    scores = []
    for parent_paper in recommentations:
        rec_key = 'recommendations' if 'recommendations' in parent_paper.keys() else 'recommentations'

        # cosine similarity of query and abstract of the recommendations 
        recommended_abstracts = [rec['abstract'] for rec in parent_paper[rec_key]]
        relevancy = context_relevancy(parent_paper['query'], recommended_abstracts)

        # keyword relevancy of the query and paper
        keyword_relevancy = k_rel_scorer.calculate_relevance_scores([parent_paper['query']], recommended_abstracts)

        # cocitation score of paper 1 and paper 2
        cocitation_score = get_citation_scores(parent_paper, parent_paper[rec_key])
        parent_paper['cocitation_score'] = cocitation_score
        parent_paper['context_relevancy'] = relevancy
        parent_paper['keyword_relevancy'] = keyword_relevancy
        scores.append(parent_paper)
        break
    return scores



def print_scores(cocitation_score=None, rel=None, keyword=None):
    if cocitation_score is not None:
        print('cocitation_score:', cocitation_score)
        print('') 
    if rel is not None: 
        print('relevance:', rel)
        print('')
    if keyword is not None:
        print('keyword_relevance:', keyword)
        print('')

In [21]:
google_embed_rec_scores = get_scores('recommended_papers_google_embed.pt', 'embeddings')

In [1]:
tf_idf_rec_scores = get_scores('data/tf_idf_recommendation.pt', 'tf_idf')

In [25]:
hybrid = get_scores('data/hybrid_recommentations.pt', 'hybrid_search')
hybrid

[{'id': 'hep-th/9711200',
  'title': 'The Large N limit of superconformal field theories and supergravity',
  'query': 'supergravity theories, branes, strings, conformal field theories, hooft limit, and anti deSitter spacetimes',
  'recommendations': [{'abstract': '  Metastable, supersymmetry-breaking configurations can be created in flux\ngeometries by placing antibranes in warped throats. Via gauge/gravity duality,\nsuch configurations should have an interpretation as supersymmetry-breaking\nstates in the dual field theory. In this paper, we perturbatively determine the\nasymptotic supergravity solutions corresponding to anti-D3-brane probes placed\nat the tip of the cascading warped deformed conifold geometry, which is dual to\nan SU(N+M) x SU(N) gauge theory. The backreaction of the antibranes has the\neffect of introducing imaginary anti-self-dual flux, squashing the compact part\nof the space and forcing the dilaton to run. Using the generalization of\nholographic renormalization

In [9]:
# scores = []
# db = get_db(index)

# for parent_paper in most_cited:
#     query = create_query(parent_paper['abstract'])
#     recommended_papers = db.similarity_search(query, k=30)
#     recommendations = [{'abstract': paper.page_content,'id': paper.metadata['id']} for paper in recommended_papers]
#     scores.append({'id': parent_paper['id'], 'title':parent_paper['title'], 'query': query, 'recommendations': recommendations, 'abstract': parent_paper['abstract']})


In [10]:
# from sklearn.preprocessing import MinMaxScaler
# score_columns = ['cocitation_score', 'relevance_score', 'keyword_relevance']
# scaler = MinMaxScaler()
# tmp_df = df.copy()
# df[score_columns] = scaler.fit_transform(df[score_columns])
# df['score'] = (df['cocitation_score'] * .3) + (df['relevance_score'] * .3) + (df['keyword_relevance'] * .4)

In [11]:
# pd.to_pickle(df, 'data/train_norm_total_score.pkl')
# pd.read_pickle('data/train_norm_total_score.pkl')

Github recommender 

In [8]:
def evalutaion(cocitation_scores, relevancy_scores, keyword_relevancy):
    '''Evaluate the metric based on its score weights
    Params: 3 scores in np or tensors format, 
    Returns: 
    evaluated_score (Mean Scores)
    cocitation_score (Mean Scores)
    relevancy_score (Mean Scores)
    keyword_relevancy '''
    co_score = sum(cocitation_scores)/len(cocitation_scores)
    rel_score = sum(relevancy_scores)/len(relevancy_scores)
    key_score = sum(keyword_relevancy)/len(keyword_relevancy)
    return co_score, rel_score, key_score

In [12]:
def normalize_and_save(path, res):
    n_scores = []
    for paper in res:
        cocitation_scores = paper['cocitation_score']
        context_relevancy = paper['context_relevancy']
        keyword_relevancy = paper['keyword_relevancy']
        scores = evalutaion(cocitation_scores=cocitation_scores,
                            relevancy_scores=context_relevancy,
                            keyword_relevancy=keyword_relevancy)
        paper['n_cocitation_score'], paper['n_context_relevancy'], paper['n_keyword_relevancy'] = scores
        n_scores.append(paper)
    torch.save(n_scores, path)
        

In [13]:
# import torch

# res = torch.load('scores/google_embed_scores.pt')
# normalize_and_save('google_embed_scores.pt', res)

# res = torch.load('scores/tf_idf_scores.pt')
# normalize_and_save('tf_idf_scores.pt', res)

# res = torch.load('scores/hybrid_scores.pt')
# normalize_and_save('hybrid_scores.pt', res)